In [2]:
import cv2
import numpy as np
import os
import pickle
import json
import time
import requests
import csv
from tqdm import tqdm
from collections import defaultdict
from ultralytics import YOLO
from sort import *


In [ ]:



class TrafficViolationSystem:
    def __init__(self, config):
        self.config = config
        self.histograms = self.load_histograms()
        self.model = YOLO('yolov8n.pt')
        self.tracker = Sort(max_age=15, min_hits=3)
        self.violations = []
        self.red_light_intervals = []
        self.current_red_interval = None
        self.frame_buffer = []
        self.plate_cache = {}
        self.tracking_summary = defaultdict(dict)

    def load_histograms(self):
        histograms = {}
        for label, path in self.config['histogram_files'].items():
            with open(path, 'rb') as f:
                histograms[label] = pickle.load(f)
        return histograms

    def calculate_histogram(self, frame):
        height, width = frame.shape[:2]
        x_center, y_center, w, h = self.config['traffic_light_bbox']
        x_min = int((x_center - w/2) * width)
        x_max = int((x_center + w/2) * width)
        y_min = int((y_center - h/2) * height)
        y_max = int((y_center + h/2) * height)
        
        roi = frame[y_min:y_max, x_min:x_max]
        hist = cv2.calcHist([roi], [0,1,2], None, [8,8,8], [0,256]*3)
        return cv2.normalize(hist, hist).flatten()

    def detect_traffic_light(self, frame):
        frame_hist = self.calculate_histogram(frame)
        best_match, best_score = None, -1
        for label, hist in self.histograms.items():
            score = cv2.compareHist(frame_hist, hist, cv2.HISTCMP_CORREL)
            if score > best_score:
                best_score, best_match = score, label
        return best_match, best_score

    def detect_vehicles(self, frame):
        results = self.model(frame)
        detections = []
        for box in results[0].boxes.data:
            x1, y1, x2, y2, conf, cls = map(float, box)
            if int(cls) == 2:  # Car class
                detections.append([x1, y1, x2, y2, conf])
        return np.array(detections) if detections else np.empty((0, 5))

    def is_within_violation_zone(self, x, y):
        x_center, y_center, w, h = self.config['violation_zone_bbox']
        x_min = int((x_center - w/2) * self.video_width)
        x_max = int((x_center + w/2) * self.video_width)
        y_min = int((y_center - h/2) * self.video_height)
        y_max = int((y_center + h/2) * self.video_height)
        return x_min <= x <= x_max and y_min <= y <= y_max

    def track_violations(self, frame, frame_number):
        detections = self.detect_vehicles(frame)
        tracked_objs = self.tracker.update(detections)
        
        for obj in tracked_objs:
            x1, y1, x2, y2, track_id = map(int, obj)
            cx, cy = (x1+x2)//2, (y1+y2)//2
            
            if not self.is_within_violation_zone(cx, cy):
                continue
            
            if track_id not in self.tracking_summary:
                self.tracking_summary[track_id] = {
                    'first_seen': frame_number,
                    'last_seen': frame_number,
                    'violation_frames': [],
                    'plate_number': 'UNKNOWN'
                }
            
            self.tracking_summary[track_id]['last_seen'] = frame_number
            self.tracking_summary[track_id]['violation_frames'].append(frame_number)
            
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0,0,255), 2)
            cv2.putText(frame, f"ID: {track_id}", (x1, y1-10), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,255), 2)

    def process_video(self):
        cap = cv2.VideoCapture(self.config['input_video'])
        self.video_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        self.video_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        fps = cap.get(cv2.CAP_PROP_FPS)
        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        
        writer = cv2.VideoWriter(self.config['output_video'], 
                               cv2.VideoWriter_fourcc(*'mp4v'), 
                               fps, (self.video_width, self.video_height))
        
        red_active = False
        for frame_num in tqdm(range(total_frames), desc="Processing Frames"):
            ret, frame = cap.read()
            if not ret: break
            
            light_status, _ = self.detect_traffic_light(frame)
            
            # Red light state management
            if light_status == 'Red' and not red_active:
                red_active = True
                self.current_red_interval = {
                    'start_frame': frame_num,
                    'start_time': frame_num/fps,
                    'end_frame': None,
                    'end_time': None
                }
                self.frame_buffer = []
            elif light_status != 'Red' and red_active:
                red_active = False
                self.current_red_interval['end_frame'] = frame_num
                self.current_red_interval['end_time'] = frame_num/fps
                self.red_light_intervals.append(self.current_red_interval)
                self.save_redlight_clip()
                self.frame_buffer = []
            
            if red_active:
                self.track_violations(frame, frame_num)
                self.frame_buffer.append(frame.copy())
            
            writer.write(frame)
        
        cap.release()
        writer.release()
        self.process_license_plates()
        self.generate_output()

    def save_redlight_clip(self):
        if not self.frame_buffer:
            return
        
        clip_path = os.path.join(
            self.config['output_folder'],
            f"redlight_{len(self.red_light_intervals)}.mp4"
        )
        
        h, w = self.frame_buffer[0].shape[:2]
        writer = cv2.VideoWriter(clip_path, cv2.VideoWriter_fourcc(*'mp4v'), 
                               self.config['output_fps'], (w, h))
        for f in self.frame_buffer:
            writer.write(f)
        writer.release()

    def process_license_plates(self):
        api_key = self.config['plate_api_key']
        for track_id, data in tqdm(self.tracking_summary.items(), desc="Processing Plates"):
            try:
                frame_num = data['violation_frames'][0]
                cap = cv2.VideoCapture(self.config['input_video'])
                cap.set(cv2.CAP_PROP_POS_FRAMES, frame_num)
                ret, frame = cap.read()
                
                if ret:
                    x1, y1, x2, y2 = data['bbox']  # Store bbox from tracking
                    cropped = frame[y1:y2, x1:x2]
                    cv2.imwrite('temp.jpg', cropped)
                    
                    plate = self.recognize_plate('temp.jpg', api_key)
                    data['plate_number'] = plate
            except Exception as e:
                print(f"Error processing track {track_id}: {str(e)}")
            finally:
                cap.release()

    def recognize_plate(self, image_path, api_key, retries=3):
        for _ in range(retries):
            try:
                with open(image_path, 'rb') as f:
                    response = requests.post(
                        'https://api.platerecognizer.com/v1/plate-reader/',
                        files={'upload': f},
                        headers={'Authorization': f'Token {api_key}'}
                    )
                if response.status_code == 200:
                    results = response.json().get('results', [])
                    return results[0]['plate'] if results else 'UNKNOWN'
            except Exception as e:
                time.sleep(2)
        return 'UNKNOWN'

    def generate_output(self):
        output_data = {
            'redlight_intervals': self.red_light_intervals,
            'violations': []
        }
        
        for track_id, data in self.tracking_summary.items():
            output_data['violations'].append({
                'track_id': track_id,
                'plate_number': data['plate_number'],
                'first_seen': data['first_seen'],
                'last_seen': data['last_seen'],
                'duration': (data['last_seen'] - data['first_seen'])/self.config['output_fps']
            })
        
        with open(self.config['json_output'], 'w') as f:
            json.dump(output_data, f, indent=2)

# %% [code]
# Configuration
config = {
    "input_video": "./vid11_27_7_FaisalTown.mp4",
    "output_video": "./Results/final_output.mp4",
    "output_folder": "./violationClips",
    "json_output": "./Results/violations.json",
    "output_fps": 25,
    "plate_api_key": "df9ea40d34e80d6149f57d04d8b1328d61920916",
    "histogram_files": {
        "Green": "./Histogram_Data/averaged_hist_Green.pkl",
        "Red": "./Histogram_Data/averaged_hist_Red.pkl",
        "Orange": "./Histogram_Data/averaged_hist_Orange.pkl",
        "None": "./Histogram_Data/averaged_hist_None.pkl"
    },
    "traffic_light_bbox": (0.485879, 0.577487, 0.016630, 0.051067),
    "violation_zone_bbox": (0.901061, 0.730515, 0.197877, 0.199434)
}

# Create output directory
os.makedirs(config['output_folder'], exist_ok=True)

# Run the system
system = TrafficViolationSystem(config)
system.process_video()

The code by Saim for clipping videos  

In [ ]:

import cv2
import numpy as np
import os
import pickle
from ultralytics import YOLO  # Assuming YOLOv8 is installed and available


def load_histogram(file_path):
    """Load a histogram from a pickle file."""
    with open(file_path, 'rb') as file:
        histogram = pickle.load(file)
    return histogram


def calculate_histogram(image, bbox):
    """Calculate and normalize the histogram for a region in the image."""
    height, width, _ = image.shape
    x_center, y_center, box_width, box_height = bbox

    # Convert normalized coordinates to pixel coordinates
    x_min = int((x_center - box_width / 2) * width)
    x_max = int((x_center + box_width / 2) * width)
    y_min = int((y_center - box_height / 2) * height)
    y_max = int((y_center + box_height / 2) * height)

    # Crop the region of interest
    roi = image[y_min:y_max, x_min:x_max]

    # Calculate the histogram
    histogram = cv2.calcHist([roi], [0, 1, 2], None, [8, 8, 8], [0, 256, 0, 256, 0, 256])
    histogram = cv2.normalize(histogram, histogram).flatten()
    return histogram


def compare_histograms(frame_histogram, histograms):
    """Compare the frame histogram with a list of histograms using correlation."""
    best_match = None
    best_score = -1

    for label, histogram in histograms.items():
        score = cv2.compareHist(frame_histogram, histogram, cv2.HISTCMP_CORREL)
        if score > best_score:
            best_score = score
            best_match = label

    return best_match, best_score


def draw_arrow(frame, x1, y1, x2, y2):
    """Draw a large, filled downward arrow above the detected object."""
    center_x = (x1 + x2) // 2
    top_y = y1 - 150  # Position above the bounding box

    # Define arrow dimensions
    arrow_width = 50
    arrow_height = 100
    shaft_width = 20

    # Coordinates for the arrowhead
    arrow_tip = (center_x, y1)
    left_corner = (center_x - arrow_width, top_y + arrow_height)
    right_corner = (center_x + arrow_width, top_y + arrow_height)

    # Coordinates for the shaft
    shaft_top_left = (center_x - shaft_width, top_y)
    shaft_top_right = (center_x + shaft_width, top_y)
    shaft_bottom_left = (center_x - shaft_width, top_y + arrow_height)
    shaft_bottom_right = (center_x + shaft_width, top_y + arrow_height)

    # Combine polygons to form the arrow
    arrow_head = np.array([arrow_tip, left_corner, right_corner], np.int32)
    arrow_shaft = np.array([shaft_top_left, shaft_bottom_left, shaft_bottom_right, shaft_top_right], np.int32)

    # Draw the filled polygons
    color = (0, 0, 255)  # Red color for the arrow
    cv2.fillPoly(frame, [arrow_head], color)
    cv2.fillPoly(frame, [arrow_shaft], color)


def detect_cars(frame, model, specified_path):
    """Detect cars only within the specified path region."""
    results = model(frame)  # Perform inference
    detections = []

    for box in results[0].boxes.data:  # Access bounding box data
        x_min, y_min, x_max, y_max = map(int, box[:4])  # Extract coordinates
        cls = int(box[5])  # Class ID
        if cls == 2:  # Class ID for 'car'
            # Check if the bounding box lies within the specified path
            car_center_x = (x_min + x_max) // 2
            car_center_y = (y_min + y_max) // 2
            if is_within_path(car_center_x, car_center_y, specified_path, frame.shape):
                detections.append((x_min, y_min, x_max, y_max))
                draw_arrow(frame, x_min, y_min, x_max, y_max)  # Draw arrow on the car
    return detections


def is_within_path(car_center_x, car_center_y, path_bbox, frame_shape):
    """Check if the car's center lies within the specified path."""
    height, width, _ = frame_shape
    x_center, y_center, box_width, box_height = path_bbox

    # Convert normalized path coordinates to pixel coordinates
    x_min = int((x_center - box_width / 2) * width)
    x_max = int((x_center + box_width / 2) * width)
    y_min = int((y_center - box_height / 2) * height)
    y_max = int((y_center + box_height / 2) * height)

    return x_min <= car_center_x <= x_max and y_min <= car_center_y <= y_max


def process_video(input_video_path, histogram_files, bbox, path_bbox, output_video_path, save_frames=False, temp_frame_folder=None):
    """
    Process video, match histograms, detect cars, and output processed video.
    Only frames with a "Red" signal are written to the output video.
    """
    # Load the precomputed histograms
    histograms = {label: load_histogram(file) for label, file in histogram_files.items()}

    # Initialize YOLO model
    model = YOLO('yolov8n.pt')  # Use YOLOv8 pretrained model

    # Open the input video
    video_cap = cv2.VideoCapture(input_video_path)
    fps = int(video_cap.get(cv2.CAP_PROP_FPS))
    frame_width = int(video_cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(video_cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    
    # Set up the video writer for only the red-signal frames
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    video_writer = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

    frame_count = 0
    saved_frame_count = 0

    while video_cap.isOpened():
        ret, frame = video_cap.read()
        if not ret:
            break

        # Calculate the histogram for the specified region
        frame_histogram = calculate_histogram(frame, bbox)

        # Compare histograms to determine the signal color
        matched_label, score = compare_histograms(frame_histogram, histograms)

        # Draw the bounding box for the region where histogram is calculated
        height, width, _ = frame.shape
        x_center, y_center, box_width, box_height = bbox
        x_min = int((x_center - box_width / 2) * width)
        x_max = int((x_center + box_width / 2) * width)
        y_min = int((y_center - box_height / 2) * height)
        y_max = int((y_center + box_height / 2) * height)

        # Color mapping for visualization (optional)
        color_map = {
            "Green": (0, 255, 0),
            "None": (255, 255, 255),
            "Orange": (0, 165, 255),
            "Red": (0, 0, 255)
        }
        color = color_map.get(matched_label, (255, 255, 255))  # Default to white if label not found
        cv2.rectangle(frame, (x_min, y_min), (x_max, y_max), color, 2)
        label_text = f"{matched_label} ({score:.2f})"
        cv2.putText(frame, label_text, (x_min, y_max + 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

        # Only process and save the frame if the signal is "Red"
        if matched_label == "Red":
            # Detect cars in the frame
            car_detections = detect_cars(frame, model, path_bbox)
            for (x1, y1, x2, y2) in car_detections:
                # Car detections already include the arrow drawing, but drawing the bounding box again if desired
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 255), 2)  # Red for car detections

            # Optionally save frames as images
            if save_frames and temp_frame_folder:
                os.makedirs(temp_frame_folder, exist_ok=True)
                temp_frame_path = os.path.join(temp_frame_folder, f"frame_{frame_count:04d}.jpg")
                cv2.imwrite(temp_frame_path, frame)

            # Write the processed frame to the output video clip
            video_writer.write(frame)
            saved_frame_count += 1

        frame_count += 1

    video_cap.release()
    video_writer.release()
    print(f"Processed video clip saved at {output_video_path}")
    print(f"Total frames processed: {frame_count}, Frames saved: {saved_frame_count}")


# Inputs
input_video_path = './vid11_27_7_FaisalTown.mp4'  # Input video path
histogram_files = {
    "Green": "./Histogram_Data/averaged_hist_Green.pkl",
    "Red": "./Histogram_Data/averaged_hist_Red.pkl",
    "Orange": "./Histogram_Data/averaged_hist_Orange.pkl",
    "None": "./Histogram_Data/averaged_hist_None.pkl"
}
bbox = (0.485879, 0.577487, 0.016630, 0.051067)  # Normalized coordinates for histogram calculation
path_bbox = (0.901061, 0.730515, 0.197877, 0.199434)  # New specified path coordinates for car detection
output_video_path = './violationClips/output_red_clip_video.mp4'  # Output video path for red-signal clip

# Process video and save only red-signal frames as a clip
process_video(input_video_path, histogram_files, bbox, path_bbox, output_video_path)


Gradio code 

In [5]:
import cv2
import numpy as np
import os
import pickle
import tempfile
from datetime import datetime
import gradio as gr
from ultralytics import YOLO  # Assuming YOLOv8 is installed and available

# Load histogram from a pickle file
def load_histogram(file_path):
    """Load a histogram from a pickle file."""
    with open(file_path, 'rb') as file:
        histogram = pickle.load(file)
    return histogram

# Calculate histogram for a region in the image
def calculate_histogram(image, bbox):
    """Calculate and normalize the histogram for a region in the image."""
    height, width, _ = image.shape
    x_center, y_center, box_width, box_height = bbox

    # Convert normalized coordinates to pixel coordinates
    x_min = int((x_center - box_width / 2) * width)
    x_max = int((x_center + box_width / 2) * width)
    y_min = int((y_center - box_height / 2) * height)
    y_max = int((y_center + box_height / 2) * height)

    # Crop the region of interest
    roi = image[y_min:y_max, x_min:x_max]

    # Calculate the histogram
    histogram = cv2.calcHist([roi], [0, 1, 2], None, [8, 8, 8], [0, 256, 0, 256, 0, 256])
    histogram = cv2.normalize(histogram, histogram).flatten()
    return histogram

# Compare histograms to determine the signal color
def compare_histograms(frame_histogram, histograms):
    """Compare the frame histogram with a list of histograms using correlation."""
    best_match = None
    best_score = -1

    for label, histogram in histograms.items():
        score = cv2.compareHist(frame_histogram, histogram, cv2.HISTCMP_CORREL)
        if score > best_score:
            best_score = score
            best_match = label

    return best_match, best_score

# Draw a large, filled downward arrow above the detected object
def draw_arrow(frame, x1, y1, x2, y2):
    """Draw a large, filled downward arrow above the detected object."""
    center_x = (x1 + x2) // 2
    top_y = y1 - 150  # Position above the bounding box

    # Define arrow dimensions
    arrow_width = 50
    arrow_height = 100
    shaft_width = 20

    # Coordinates for the arrowhead
    arrow_tip = (center_x, y1)
    left_corner = (center_x - arrow_width, top_y + arrow_height)
    right_corner = (center_x + arrow_width, top_y + arrow_height)

    # Coordinates for the shaft
    shaft_top_left = (center_x - shaft_width, top_y)
    shaft_top_right = (center_x + shaft_width, top_y)
    shaft_bottom_left = (center_x - shaft_width, top_y + arrow_height)
    shaft_bottom_right = (center_x + shaft_width, top_y + arrow_height)

    # Combine polygons to form the arrow
    arrow_head = np.array([arrow_tip, left_corner, right_corner], np.int32)
    arrow_shaft = np.array([shaft_top_left, shaft_bottom_left, shaft_bottom_right, shaft_top_right], np.int32)

    # Draw the filled polygons
    color = (0, 0, 255)  # Red color for the arrow
    cv2.fillPoly(frame, [arrow_head], color)
    cv2.fillPoly(frame, [arrow_shaft], color)

# Detect cars within the specified path region
def detect_cars(frame, model, specified_path):
    """Detect cars only within the specified path region."""
    results = model(frame)  # Perform inference
    detections = []

    for box in results[0].boxes.data:  # Access bounding box data
        x_min, y_min, x_max, y_max = map(int, box[:4])  # Extract coordinates
        cls = int(box[5])  # Class ID
        if cls == 2:  # Class ID for 'car'
            # Check if the bounding box lies within the specified path
            car_center_x = (x_min + x_max) // 2
            car_center_y = (y_min + y_max) // 2
            if is_within_path(car_center_x, car_center_y, specified_path, frame.shape):
                detections.append((x_min, y_min, x_max, y_max))
                draw_arrow(frame, x_min, y_min, x_max, y_max)  # Draw arrow on the car
    return detections

# Check if a car's center lies within the specified path
def is_within_path(car_center_x, car_center_y, path_bbox, frame_shape):
    """Check if the car's center lies within the specified path."""
    height, width, _ = frame_shape
    x_center, y_center, box_width, box_height = path_bbox

    # Convert normalized path coordinates to pixel coordinates
    x_min = int((x_center - box_width / 2) * width)
    x_max = int((x_center + box_width / 2) * width)
    y_min = int((y_center - box_height / 2) * height)
    y_max = int((y_center + box_height / 2) * height)

    return x_min <= car_center_x <= x_max and y_min <= car_center_y <= y_max

# Process video and generate output
def process_video(input_video_path, histogram_files, bbox, path_bbox, output_video_path):
    """
    Process video, match histograms, detect cars, and output processed video.
    Only frames with a "Red" signal are written to the output video.
    """
    # Load the precomputed histograms
    histograms = {label: load_histogram(file) for label, file in histogram_files.items()}

    # Initialize YOLO model
    model = YOLO('yolov8n.pt')  # Use YOLOv8 pretrained model

    # Open the input video
    video_cap = cv2.VideoCapture(input_video_path)
    fps = int(video_cap.get(cv2.CAP_PROP_FPS))
    frame_width = int(video_cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(video_cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    
    # Set up the video writer for only the red-signal frames
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    video_writer = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

    frame_count = 0
    saved_frame_count = 0

    while video_cap.isOpened():
        ret, frame = video_cap.read()
        if not ret:
            break

        # Calculate the histogram for the specified region
        frame_histogram = calculate_histogram(frame, bbox)

        # Compare histograms to determine the signal color
        matched_label, score = compare_histograms(frame_histogram, histograms)

        # Draw the bounding box for the region where histogram is calculated
        height, width, _ = frame.shape
        x_center, y_center, box_width, box_height = bbox
        x_min = int((x_center - box_width / 2) * width)
        x_max = int((x_center + box_width / 2) * width)
        y_min = int((y_center - box_height / 2) * height)
        y_max = int((y_center + box_height / 2) * height)

        # Color mapping for visualization
        color_map = {
            "Green": (0, 255, 0),
            "None": (255, 255, 255),
            "Orange": (0, 165, 255),
            "Red": (0, 0, 255)
        }
        color = color_map.get(matched_label, (255, 255, 255))  # Default to white if label not found
        cv2.rectangle(frame, (x_min, y_min), (x_max, y_max), color, 2)
        label_text = f"{matched_label} ({score:.2f})"
        cv2.putText(frame, label_text, (x_min, y_max + 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

        # Only process and save the frame if the signal is "Red"
        if matched_label == "Red":
            # Detect cars in the frame
            car_detections = detect_cars(frame, model, path_bbox)
            for (x1, y1, x2, y2) in car_detections:
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 255), 2)  # Red for car detections

            # Write the processed frame to the output video clip
            video_writer.write(frame)
            saved_frame_count += 1

        frame_count += 1

    video_cap.release()
    video_writer.release()
    print(f"Processed video clip saved at {output_video_path}")
    print(f"Total frames processed: {frame_count}, Frames saved: {saved_frame_count}")

# Gradio wrapper function
def gradio_process_video(input_video):
    """Wrapper function for Gradio interface."""
    # Define the output video path
    output_video_path = "./red_output_video.mp4"
    
    # Define parameters
    histogram_files = {
        "Green": "./Histogram_Data/averaged_hist_Green.pkl",
        "Red": "./Histogram_Data/averaged_hist_Red.pkl",
        "Orange": "./Histogram_Data/averaged_hist_Orange.pkl",
        "None": "./Histogram_Data/averaged_hist_None.pkl"
    }
    bbox = (0.485879, 0.577487, 0.016630, 0.051067)  # Normalized coordinates for histogram calculation
    path_bbox = (0.901061, 0.730515, 0.197877, 0.199434)  # Path coordinates for car detection

    # Process the video
    process_video(
        input_video_path=input_video,
        histogram_files=histogram_files,
        bbox=bbox,
        path_bbox=path_bbox,
        output_video_path=output_video_path
    )
    
    return output_video_path

# Gradio UI
with gr.Blocks(title="Automatic Traffic Violation Detection") as demo:
    gr.Markdown("# 🚦 Automatic Traffic Violation Detection System")
    
    with gr.Row():
        with gr.Column():
            input_video = gr.Video(label="Upload Traffic Video", sources=["upload"])
            process_btn = gr.Button("Detect Violations", variant="primary")
        
        with gr.Column():
            output_video = gr.Video(label="Violations Clip (Red Light Period)", autoplay=True)
    
    process_btn.click(
        fn=gradio_process_video,
        inputs=[input_video],
        outputs=[output_video],
        
    )

if __name__ == "__main__":
    demo.launch(
        share = True
    )
    

* Running on local URL:  http://127.0.0.1:7863
* Running on public URL: https://20467e26444be67412.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)



0: 384x640 11 persons, 9 cars, 5 motorcycles, 1 truck, 80.4ms
Speed: 6.2ms preprocess, 80.4ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 7 persons, 9 cars, 3 motorcycles, 1 truck, 80.8ms
Speed: 6.5ms preprocess, 80.8ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 10 persons, 10 cars, 5 motorcycles, 169.4ms
Speed: 4.3ms preprocess, 169.4ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 6 persons, 11 cars, 3 motorcycles, 112.3ms
Speed: 3.9ms preprocess, 112.3ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 6 persons, 7 cars, 3 motorcycles, 178.3ms
Speed: 4.1ms preprocess, 178.3ms inference, 2.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 11 persons, 9 cars, 3 motorcycles, 182.6ms
Speed: 10.7ms preprocess, 182.6ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 12 persons, 7 cars, 3 motorcycles, 1 truck, 91.2ms
Speed